<a href="https://colab.research.google.com/github/Sanmuga/Image-caption-generator/blob/main/Image_caption_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
pip install tensorflow

In [17]:
pip install numpy pandas

In [18]:
pip install Pillow

In [19]:
pip install os-sys

  Using cached os_sys-2.1.4-py3-none-any.whl (15.6 MB)
  Using cached pygubu-0.31-py3-none-any.whl (118 kB)
  Using cached progress-1.6.tar.gz (7.8 kB)
  Preparing metadata (setup.py) ... done
  Using cached progressbar-2.5.tar.gz (10 kB)
  Preparing metadata (setup.py) ... done
  Using cached jupyter-1.0.0-py2.py3-none-any.whl (2.7 kB)
  Using cached Eel-0.16.0.tar.gz (24 kB)
  Preparing metadata (setup.py) ... done
  Using cached extract_zip-1.0.0-py3-none-any.whl (2.6 kB)
INFO: pip is looking at multiple versions of os-sys to determine which version is compatible with other requirements. This could take a while.
  Using cached os_sys-2.1.3-py3-none-any.whl (15.5 MB)
  Using cached os_sys-2.1.2-py3-none-any.whl (15.4 MB)
  Using cached os_sys-2.1.1-py3-none-any.whl (15.4 MB)
  Using cached os_sys-2.1.0-py3-none-any.whl (15.9 MB)
  Using cached os_sys-2.0.9-py3-none-any.whl (15.4 MB)
  Using cached os_sys-2.0.8-py3-none-any.whl (15.4 MB)
  Using cached os_sys-2.0.7-py3-none-any.whl (1

In [20]:
pip install numpy Pillow tensorflow tqdm

In [21]:
!pip install numpy pillow keras tensorflow

In [26]:
!pip install numpy
!pip install pillow
!pip install tensorflow
!pip install keras


In [35]:
import os
import numpy as np
from PIL import Image
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Input, Dense, Embedding, LSTM, add

# Load image features
def load_image_features(images_folder, img_ids):
    image_model = InceptionV3(weights='imagenet')
    image_model = Model(inputs=image_model.input, outputs=image_model.layers[-2].output)

    img_features = {}
    for img_id in img_ids:
        img_path = os.path.join(images_folder, img_id + '.jpg')
        img = Image.open(img_path)
        img = img.resize((299, 299))
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        features = image_model.predict(img)
        img_features[img_id] = features
    return img_features

# Load and preprocess captions
def load_captions(captions_file, img_ids):
    caption_dict = {}
    with open(captions_file, 'r') as file:
        for line in file.readlines():
            parts = line.strip().split('\t')
            img_id = parts[0].split('.')[0]
            if img_id in img_ids:
                caption = parts[1]
                if img_id not in caption_dict:
                    caption_dict[img_id] = []
                caption_dict[img_id].append(caption)
    return caption_dict

# Generate sequences
def generate_sequences(caption_dict, img_features, tokenizer, max_length, vocab_size):
    X1, X2, y = [], [], []
    for img_id, captions in caption_dict.items():
        for caption in captions:
            seq = tokenizer.texts_to_sequences([caption])[0]
            for i in range(1, len(seq)):
                in_seq, out_seq = seq[:i], seq[i]
                in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                X1.append(img_features[img_id][0])
                X2.append(in_seq)
                y.append(out_seq)
    return np.array(X1), np.array(X2), np.array(y)

# Path to the Flickr8k dataset
images_folder = '/content/drive/MyDrive/Flickr8k_Dataset/Flicker8k_Dataset'
captions_file = '/content/drive/MyDrive/Flickr8k_text/Flickr8k.token.txt'

# Load a subset of 250 images
img_ids = [filename.split('.')[0] for filename in os.listdir(images_folder)][:250]

# Load and preprocess captions for the subset
caption_dict = load_captions(captions_file, img_ids)

# Create a tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(caption_dict.values()))
vocab_size = len(tokenizer.word_index) + 1

# Load image features for the subset
img_features = load_image_features(images_folder, img_ids)


# Generate training data
max_length = max(len(seq.split()) for captions in caption_dict.values() for seq in captions)
X1, X2, y = generate_sequences(caption_dict, img_features, tokenizer, max_length, vocab_size)

# Define the model
input_img_features = Input(shape=(2048,))
img_dense = Dense(256, activation='relu')(input_img_features)

input_caption = Input(shape=(max_length,))
caption_embed = Embedding(vocab_size, 256, mask_zero=True)(input_caption)
caption_lstm = LSTM(256)(caption_embed)

decoder_input = add([img_dense, caption_lstm])
output = Dense(vocab_size, activation='softmax')(decoder_input)

model = Model(inputs=[input_img_features, input_caption], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Train the model
model.fit([X1, X2], y, epochs=10, batch_size=64)

# Function to generate caption for input image
def generate_caption_for_image(image_path, model, tokenizer, max_length):
    img = Image.open(image_path)
    img = img.resize((299, 299))
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    img_feature = model.layers[2](img)

    in_text = 'startseq'
    for _ in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([img_feature, sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = tokenizer.index_word[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    return in_text

# Example usage
image_path = '/content/sample.png'
caption = generate_caption_for_image(image_path, model, tokenizer, max_length)
print("Generated Caption:", caption)


1/1 [==============================] - 0s 219ms/step
Epoch 1/10


ValueError: ignored